In [21]:
#!/usr/bin/env python

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [22]:
# Set data

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target
N = Y.size
Y2 = np.zeros(3 * N).reshape(N,3).astype(np.float32)
for i in range(N):
    Y2[i,Y[i]] = 1.0

index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest = X[index[index % 2 == 0],:]
yans = Y[index[index % 2 == 0]]

In [23]:
# Define model

class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            # 入力4 出力6
            l1=L.Linear(4,6),
            # 入力6 出力3
            l2=L.Linear(6,3),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)

    def fwd(self,x):
         h1 = F.sigmoid(self.l1(x))
         h2 = self.l2(h1)
         return h2

In [24]:
# Initialize model

model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [25]:
# Learn

for i in range(10000):
    x = Variable(xtrain)
    y = Variable(ytrain)
    model.zerograds()
    loss = model(x,y)
    loss.backward()
    optimizer.update()

In [26]:
# Test

xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)

ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print( ok, "/", nrow, " = ", (ok * 1.0)/nrow)

[ 1.06201911  0.06794003 -0.1050007 ] 0
[ 1.02131701  0.07757384 -0.09967777] 0
[ 1.04751742  0.05147907 -0.09929588] 0
[ 0.9537459   0.05485469 -0.06293401] 0
[ 0.91575021  0.13680983 -0.05875739] 0
[ 1.08100688  0.06934446 -0.10517094] 0
[ 1.01144481  0.13902041 -0.09581015] 0
[ 1.22020543  0.00995183 -0.16934836] 0
[ 1.1184516  -0.01385289 -0.1158331 ] 0
[ 1.04656053  0.09160882 -0.0767006 ] 0
[ 0.99757558  0.15131387 -0.06552547] 0
[ 1.10296714 -0.02782667 -0.14641669] 0
[ 0.79281265  0.19783685  0.02485088] 0
[ 0.93339252  0.09815654 -0.03408006] 0
[ 1.07570612  0.08382604 -0.11070901] 0
[ 0.91734189  0.16250661 -0.04252282] 0
[ 1.0712074   0.02279142 -0.10415381] 0
[ 0.99385005  0.14950937 -0.08250415] 0
[ 1.15286624  0.06407076 -0.14403999] 0
[ 0.95961887  0.09853843 -0.08123431] 0
[ 1.06444263  0.03425357 -0.10597354] 0
[ 0.96467716  0.06952733 -0.08282322] 0
[ 0.8358441   0.11411065  0.02446249] 0
[ 0.9972989   0.06667438 -0.06743646] 0
[ 1.06513596  0.06630236 -0.09905037] 0


In [85]:
# xa = Variable(np.array(xtest), volatile='on')
#test = np.array([[ 5.099 ,  3.5       ,  1.3999,  0.2       ]]).astype(np.float32())
#xa = Variable(np.array(np.array(test)), volatile='on')
test_index = 45
xa = Variable(np.array([xtest[test_index]]).astype(np.float32()), volatile='on')
ya = model.fwd(xa)

print(ya.data)
print(yans[test_index])

[[ 0.01255564  0.5177477   0.49540833]]
1
